# Advent Of Code 2022

In [2]:
using System.IO;

## Day 1

In [2]:
IEnumerable<int> ReadCalories() {
    var total = 0;
    foreach(var line in File.ReadLines("../data/in-1.txt")) {
        if(int.TryParse(line, out var value)) {
            total += value;
        }
        else {
            yield return total;
            total = 0;
        }
    }
    yield return total;
}

Console.WriteLine(ReadCalories().Max());

69289


# Day 3

In [10]:
public class Day3 {

    int GetPriority(string str) 
    {
        long contains = 0;
        var half = str.Length / 2;
        for(var i=0; i<half; i++) 
        {
            var prio = GetPriority(str[i]);
            var mask = 1L << prio;
            contains |= mask;
        }
        for(var i=0; i<half; i++) 
        {
            var prio = GetPriority(str[i + half]);
            var mask = 1L << prio;
            if((contains & mask) > 1) {
                return prio;
            }
        }
        return 0;
    }
    
    int GetPriority(char c) 
    {
        if(c > 'Z') 
        {
            return (int)(c - 'a') + 1;
        }
        return (int)(c - 'A') + 27;
    }
    
    int Run3_1(string file) {
        return
            File
            .ReadLines(file)
            .Select(l => GetPriority(l))
            .Sum();
    }

    int GetPriority(string str1, string str2, string str3) 
    {
        var chars = str1.Distinct().Concat(str2.Distinct()).Concat(str3.Distinct());
        var c = 
            chars
            .GroupBy(c => c)
            .Single(g => g.Count() == 3)
            .Key;
        return GetPriority(c);
    }
    
    
    IEnumerable<int> Run3_2(string file) {
        var lines = File.ReadLines(file).ToList();
        for(var i=0; i<lines.Count; i+=3) {
            yield return GetPriority(lines[i], lines[i+1], lines[i+2]);
        }
    }
    

    public void Run() {
        Console.WriteLine(Run3_1("../data/in-3.txt"));
        Console.WriteLine(Run3_2("../data/in-3.txt").Sum());
    }
}

new Day3().Run();


7763
2569


# Day 4

In [18]:
public class Day4 {
    public class Range {
        public Range(int from, int to) {
            if(to < from) {
                throw new ArgumentException();
            }
            this.From = from;
            this.To = to;
        }

        public int From;
        public int To;

        public bool Contains(int n) => From <= n && n <= To;

        public bool FullyContains(Range other) => Contains(other.From) && Contains(other.To);

        public bool Overlaps(Range other) {
            if(other.To < this.From) {
                return false;
            }
            if(other.From > this.To) {
                return false;
            }
            return true;
        }

        public static Range Parse(string str) {
            var parts = str.Split('-');
            return new Range(int.Parse(parts[0]), int.Parse(parts[1]));
        }
    }

    public int Run1(string file) {
        var count = 0;
        foreach(var line in File.ReadAllLines(file)) {
            var parts = line.Split(',');
            var range1 = Range.Parse(parts[0]);
            var range2 = Range.Parse(parts[1]);
            if(range1.FullyContains(range2) || range2.FullyContains(range1)) {
                count++;
            }
        }
        return count;
    }

    public int Run2(string file) {
        var count = 0;
        foreach(var line in File.ReadAllLines(file)) {
            var parts = line.Split(',');
            var range1 = Range.Parse(parts[0]);
            var range2 = Range.Parse(parts[1]);
            if(range1.Overlaps(range2)) {
                count++;
            }
        }
        return count;
    }


    public void Run() {
        Console.WriteLine(Run1("../data/in-4.txt"));
        Console.WriteLine(Run2("../data/in-4.txt"));
    }
}

new Day4().Run();

651
956


# Day 5


In [14]:
public class Day5 {

    public class Yard {
        public Yard(int size) {
            for(var i=0; i<size; i++)  {
                Stacks.Add(new());
            }
        }
        public List<LinkedList<char>> Stacks = new ();

        public void Move(Move move, int mode) {
            if(mode == 9000) {
                Move9000(move);
            }
            if(mode == 9001) {
                Move9001(move);
            }
        }

        public void Move9000(Move move) {
            for(var i=0; i<move.Quantity; i++) {
                var item = Stacks[move.From].First.Value;
                Stacks[move.From].RemoveFirst();
                Stacks[move.To].AddFirst(item);
            }
        }

        public void Move9001(Move move) {
            var items = new List<char>();            
            for(var i=0; i<move.Quantity; i++) {
                var item = Stacks[move.From].First.Value;
                Stacks[move.From].RemoveFirst();
                items.Add(item);
            }
            for(var i=0; i<move.Quantity; i++) {
                Stacks[move.To].AddFirst(items[items.Count - i - 1]);
            }
        }

        public void Print() {
            for(var i=0; i<Stacks.Count; i++) {
                Console.Write($"{i + 1}");
                foreach(var c in Stacks[i].Reverse()) {
                    Console.Write($" [{c}]");
                }
                Console.WriteLine();
            }
        }

        public string State() {
            var result = new StringBuilder();
            foreach(var stack in Stacks) {
                if(stack.Any()) {
                    result.Append(stack.First());
                }
            }
            return result.ToString();
        }
    }

    public class Move {
        public int Quantity;
        public int From;
        public int To;
        public static Move Parse(string str) {
            var parts = str.Split(' ');
            return new Move() {
                Quantity = int.Parse(parts[1]),
                From = int.Parse(parts[3]) - 1,
                To = int.Parse(parts[5]) - 1,
            };
        }
    }

    public void Run(string file, int mode) {
        var lines = File.ReadAllLines(file);
        var i = 0;
        string line;
        do {
            line = lines[i++];
        } while(!string.IsNullOrEmpty(line));
        var size = lines[0].Length;
        var yard = new Yard(size / 3);
        for(var l = 0; l < i - 1; l++) {
            for(var x = 0; x < size; x++) {
                var index = x * 4 + 1;
                if(index < lines[l].Length) {
                    var c = lines[l][index];
                    if(c >= 'A' && c <= 'Z') {
                        yard.Stacks[x].AddLast(c);
                    }
                }
            }
        }
        yard.Print();
        Console.WriteLine();
        
        while(i < lines.Length) {
            line = lines[i++];
            var move = Move.Parse(line);
            yard.Move(move, mode);
        }
        yard.Print();
        Console.WriteLine();
        Console.WriteLine($"End state: {yard.State()}");
        Console.WriteLine();
    }


}

//new Day5().Run("../data/in-5.txt", 9000);
new Day5().Run("../data/in-5.txt", 9001);


1 [Q] [S] [W] [C] [Z] [V] [F] [T]
2 [Q] [R] [B]
3 [B] [Z] [T] [Q] [P] [M] [S]
4 [D] [V] [F] [R] [Q] [H]
5 [J] [G] [L] [D] [B] [S] [T] [P]
6 [W] [R] [T] [Z]
7 [H] [Q] [M] [N] [S] [F] [R] [J]
8 [R] [N] [F] [H] [W]
9 [J] [Z] [T] [Q] [P] [R] [B]
10
11

1 [T]
2 [D]
3 [B] [N] [F] [R] [R] [V] [B] [S] [G]
4 [R] [J]
5 [F] [M] [S] [Q]
6 [W] [Q] [B] [J] [Z] [R] [H] [T]
7 [B] [M] [Q] [F] [J] [Z] [S] [Q] [Q] [T] [D] [T] [V] [N] [P] [R] [P] [H] [T] [W] [F] [C] [P] [H] [W] [R] [Z]
8 [S]
9 [Q] [Z] [L]
10
11

End state: TDGJQTZSL

